# SKA SDP Spectral Line Imaging Pipeline

+ [Overview of the pipeline](#Overview-of-the-pipeline)
+ [Pipeline stages](#Pipeline-stages)
+ [Simulate a small SKA-Mid data set](#Simulate-a-small-SKA-Mid-data-set)
+ [Run the pipeline](#Run-the-pipeline)
  - [Setting up the config file](#Setting-up-the-config-file)
  - [Execute the pipeline](#Execute-the-pipeline)

## Overview of the pipeline

The SKA SDP spectral line imaging pipeline ([source code](https://gitlab.com/ska-telescope/sdp/science-pipeline-workflows/ska-sdp-spectral-line-imaging), [documentation](https://developer.skao.int/projects/ska-sdp-spectral-line-imaging/en/latest/)) produces channel maps, with an optional continuum subtraction step. 



## Pipeline stages

This section summarizes the different processing stages currently implemented in the pipeline. There is a one-to-one mapping between these pipeline stages and the parameter definition in the yaml file. Parameters relevant to each of the pipeline stage are described on [this page](https://developer.skao.int/projects/ska-sdp-spectral-line-imaging/en/latest/stage_config.html).

+ load_data - allows users to do some basic data selection and filtering.
+ vis_stokes_conversion - performs any necessary Stokes conversion
+ read_model - If continuum subtraction needs to be done, this stage handles the continuum model that is specified as a FITS image.
+ predict_stage - If continuum subtraction is requested, this stage converts the input FITS file into model visibilities.
+ continuum_subtraction - This stage subtracts the model visibilities predicted in the previous step from the input visibility data.
+ flagging - Performs an optional RFI flagging
+ imaging - Produces the channel maps. 

## Simulate a small SKA-Mid data set

Let's first simulate a small SKA-Mid dataset to test the spectral line imaging pipeline. 

Since we will use CASA to generate the mock dataset, first, setup the measures path correctly.

In [1]:
import os
from casaconfig import config, measures_update, pull_data, set_casacore_path

measures_path = "casa_data"
try:
    os.mkdir(measures_path)
except FileExistsError:
    pass

pull_data(measures_path)
config.measurespath = measures_path
set_casacore_path(measures_path)
measures_update()

pull_data: version is already at the expected version and force is False. Nothing was changed
writing /root/.casarc...
measures_update: version installed or checked less than 1 day ago, nothing updated or checked


found existing .casarc...


Next, simulate a dummy SKA-Mid AA* measurement set with 4 channels and 1 time resolution element. 

In [2]:
from casatools import simulator, measures
from casatasks import listobs
from casatasks.private import simutil
import shutil

sm = simulator()
su = simutil.simutil()
me = measures()

input_ms_name = "spectral_imager_input.ms"
try:
    shutil.rmtree(input_ms_name)
except FileNotFoundError:
    pass
sm.open(ms=input_ms_name)

# Set the antenna configuration
ant_config = "../test_data/mid_aastar.txt"
(x, y, z, d, an, an2, telname, obspos) = su.readantenna(ant_config)
sm.setconfig(
    telescopename=telname,
    x=x,
    y=y,
    z=z,
    dishdiameter=d,
    mount=["alt-az"],
    antname=an,
    coordsystem="global",
    referencelocation=obspos,
)

# Setup the polarization mode
sm.setfeed(mode="perfect X Y", pol=[""])

# Spectral window setup
sm.setspwindow(
    spwname="Band 2",
    freq="1.4GHz",
    freqresolution="13.44kHz",
    deltafreq="13.44kHz",
    nchannels=4,
    stokes="XX YY",
)

# Specify the fields to observe
sm.setfield(
    sourcename="PKS1934-63",
    sourcedirection=me.direction(rf="J2000", v0="19h39m25.0261s", v1="-63d42m45.625s"),
)

# Define integration time
sm.settimes(
    integrationtime="10s",
    usehourangle=True,
    referencetime=me.epoch("UTC", "2019/10/4/00:00:00"),
)

# Construct the measurement set and fill in the meta data
sm.observe(sourcename="PKS1934-63", spwname="Band 2", starttime="-10s", stoptime="+10s")

sm.close();

Next, create a skymodel containing two point sources with different spectral indices. 

In [3]:
from casatools import componentlist

cl = componentlist()

model_file = "model.cl"
try:
    shutil.rmtree(model_file)
except FileNotFoundError:
    pass

# First component at the pointing center
cl.addcomponent(
    dir="J2000 19h39m25.0261s -63d42m45.625s",
    flux=1.0,
    fluxunit="Jy",
    freq="1.4GHz",
    shape="point",
    spectrumtype="spectral index",
    index=0.1,
)
# Second component is an arcmin away
cl.addcomponent(
    dir="J2000 19h39m25.0261s -63d41m45.625s",
    flux=1.0,
    fluxunit="Jy",
    freq="1.4GHz",
    shape="point",
    spectrumtype="spectral index",
    index=0.1,
)
cl.rename(filename=model_file)
cl.done();

Predict the model into the measurement set.

In [4]:
from casatasks import ft
from casatools import table
ft(vis=input_ms_name, complist=model_file, incremental=False, usescratch=True)
# ft writes the visibilities to MODEL_DATA. Copy it to DATA and delete MODEL_DATA
tb = table()
tb.open(input_ms_name, nomodify=False)
data=tb.getcol("MODEL_DATA")
tb.putcol("DATA", data)
tb.removecols("MODEL_DATA")
tb.removecols("CORRECTED_DATA")
tb.flush()
tb.close();

Convert the visibility data to MSv4 format. 

In [ ]:
from xradio.vis import convert_msv2_to_processing_set

convert_msv2_to_processing_set(
    in_file="spectral_imager_input.ms", 
    out_file="spectral_imager_input.zarr", 
    overwrite=True
)

## Running the pipeline

We can access the pipeline either via the command line executable `spectral-line-imaging-pipeline` or using the API. In this section, we will look at how to run the pipeline from the command line. 

### Setting up the config file

The input to `spectral-line-imaging-pipeline` is supplied using a config file. The easiest way to create the config file is to run 

`spectral-line-imaging-pipeline install-config --config-install-path ./` 

This creates a file named `spectral_line_imaging_pipeline.yml` in the directory mentioned above. The config file has three sections:
+ **global parameters** section which is currently not in use
+ **pipeline** section which is defines the pipeline stages to run
+ **parameters** section is where the setup of each pipeline stage defined above is specified.

As a first test run, I want to make a dirty Stokes I channel cube from the mock visibility dataset we simulated above. So, I've renamed the config file to `spectral_line_dirty.yml` and made the following changes:
1. Disabled continuum subtraction and flagging by setting the following pipeline stages to `false`: `continuum_subtraction`, `predict_stage`, `flagging`, and `read_model`. I've also removed the corresponding sections from the **parameters** section. This is not strictly necessary but it makes the config file easy to read. 
2. Disable deconvolution by setting `parameters.n_iter_major: 0`. I've also removed the `deconvolution_params` block.
3. Edit `vis_stokes_conversion.output_polarizations` and run 

### Execute the pipeline

Next, run the pipeline.

In [5]:
!spectral-line-imaging-pipeline run --input spectral_imager_input.zarr --config spectral_line_dirty.yml

1|2024-11-20T10:20:53.930Z|INFO|MainThread|run|pipeline.py#201||=============== START =====================
1|2024-11-20T10:20:53.930Z|INFO|MainThread|run|pipeline.py#202||Executing spectral_line_imaging_pipeline pipeline with metadata:
1|2024-11-20T10:20:53.931Z|INFO|MainThread|run|pipeline.py#203||Infile Path: spectral_imager_input.zarr
1|2024-11-20T10:20:53.931Z|INFO|MainThread|run|pipeline.py#204||Stages: ['load_data', 'vis_stokes_conversion', 'imaging']
1|2024-11-20T10:20:53.931Z|INFO|MainThread|run|pipeline.py#205||Configuration Path: spectral_line_dirty.yml
1|2024-11-20T10:20:53.931Z|INFO|MainThread|run|pipeline.py#206||Current run output path : ./output/spectral_line_imaging_pipeline_2024-11-20T10:20:53
1|2024-11-20T10:20:53.932Z|INFO|MainThread|run|pipeline.py#219||Selected stages to run: load_data, vis_stokes_conversion, imaging
1|2024-11-20T10:20:54.067Z|INFO|MainThread|imaging_stage|imaging.py#196||Estimating cell size...
1|2024-11-20T10:20:54.103Z|INFO|MainThread|imaging_s

If the pipeline runs successfully, you should see a new directory `output/spectral_line_imaging_pipeline_<time stamp>`, which contains the spectral cube along with pipeline log and a copy of the input config file. 